In [26]:
import numpy as np
import tensorflow as tf
from utils.draw import draw_single
print(f'Tensorflow v{tf.__version__}')

In [27]:
def generate(count):
    X = np.random.randint(0, high=256, size=(count, 9))
    a = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0], 
                  [0, 0, 0, 1, 1, 1, 0, 0, 0], 
                  [0, 0, 0, 0, 0, 0, 1, 1, 1]])

    Y = np.argmax(X.dot(a.T), axis=1)
    return X, Y

In [28]:
batch = 500
epochs = 10

def convert(x, y): return tf.dtypes.cast(x, tf.float32)/255., tf.one_hot(y, depth=3)

train_ds = tf.data.Dataset.from_tensor_slices(generate(60000))
train_ds = train_ds.map(convert)
train_ds = train_ds.shuffle(buffer_size=1000)
train_ds = train_ds.batch(batch)
train_ds = train_ds.repeat(epochs)

# Setting things up!

In [29]:
class MySquareModel(tf.keras.layers.Layer):

    def __init__(self):
        super(MySquareModel, self).__init__()
        self.w = self.add_weight(shape=(9, 3),
                                initializer='random_uniform',
                                trainable=True)
        self.b = self.add_weight(shape=(3,),
                                initializer='zeros',
                                trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [38]:
model = tf.keras.models.Sequential([MySquareModel()])
img = tf.constant([np.array([123, 245, 145, 12, 134, 22, 255, 43, 231]) / 255.], dtype='float32')
model(img)

In [39]:
model.trainable_variables

# The Optimization Loop

In [30]:
# set up all teh things
cost = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=.01)

model = tf.keras.models.Sequential([MySquareModel()])

train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.Accuracy()

In [31]:
i = 0
for X, Y in train_ds:
    with tf.GradientTape() as tape:
        pred = model(X)
        loss = cost(pred, Y)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(tf.math.argmax(Y, axis=1), tf.math.argmax(logits, axis=1))
    i+=1
    if i % 50 == 0:
        print(f'Loss: {train_loss.result()}, Accuracy: {train_accuracy.result()}')

# Trying it out (inference)

In [40]:
labels = ['top', 'middle', 'bottom']
img = tf.constant([np.array([123, 245, 145, 12, 134, 22, 255, 43, 231]) / 255.], dtype='float32')

In [41]:
Y = model(img)
answer = tf.math.argmax(Y, axis=1).numpy()[0]
labels[answer]